In [ ]:
# %% [markdown]
# # Technical Data Preprocessing
# Cleans and processes raw technical data (OHLC prices) into analysis-ready format

# %%
import pandas as pd
import re
from pathlib import Path
import os

# Create processed folder if needed
os.makedirs("../data/processed/technical", exist_ok=True)

# %%
def process_technical_file(ticker):
    """
    Processes raw technical data files into cleaned versions with indicators
    Handles multiple file formats (CSV, TSV) and data quirks
    """
    raw_path = Path(f"../data/raw/technical/{ticker}_technical.csv")
    clean_path = Path(f"../data/processed/technical/{ticker}_technical_clean.csv")
    
    try:
        # Read raw file as text
        with open(raw_path, 'r', encoding='utf-8-sig') as f:
            lines = [line.strip() for line in f if line.strip()]
        
        # Parse lines
        parsed = []
        for line in lines:
            # Handle both CSV and tab-delimited formats
            if '\t' in line:
                parts = line.split('\t')
            else:
                parts = line.split(',')
            
            # Clean each value (remove surrounding quotes)
            cleaned = [re.sub(r'^"|"$', '', x).strip() for x in parts]
            parsed.append(cleaned)
        
        # Create DataFrame
        df = pd.DataFrame(parsed[1:], columns=parsed[0])
        
        # Standardize column names
        col_map = {
            'Price': 'Close',
            'Vol.': 'Volume',
            'Change %': 'Daily_Return',
            'Close': 'Close',  # Handles case where already named Close
            'Volume': 'Volume' # Handles case where already named Volume
        }
        df = df.rename(columns={col: col_map.get(col, col) for col in df.columns})
        
        # Convert data types
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
        df['Volume'] = df['Volume'].str.replace('M','').astype(float) * 1e6
        df['Daily_Return'] = df['Daily_Return'].str.replace('%','').astype(float)/100
        for col in ['Close', 'Open', 'High', 'Low']:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        
        # Calculate technical indicators
        df['SMA_20'] = df['Close'].rolling(20, min_periods=1).mean()
        df['RSI_14'] = 100 - (100 / (1 + (
            df['Close'].diff().clip(lower=0).rolling(14, min_periods=1).mean() / 
            df['Close'].diff().clip(upper=0).abs().rolling(14, min_periods=1).mean()
        )))
        
        # Save cleaned data
        df.to_csv(clean_path, index=False)
        print(f"✅ {ticker}: Processed {len(df)} rows | Saved to {clean_path}")
        return df
        
    except Exception as e:
        print(f"❌ {ticker} failed: {str(e)}")
        return None

# %%
# Process all stocks
tickers = ['AAPL', 'JPM', 'AMZN', 'PFE', 'XOM']
for ticker in tickers:
    process_technical_file(ticker)

# %%
# Verification
print("\nVerification:")
for ticker in tickers:
    path = f"../data/processed/technical/{ticker}_technical_clean.csv"
    if os.path.exists(path):
        df = pd.read_csv(path)
        print(f"{ticker}: {len(df)} rows | Columns: {list(df.columns)}")
    else:
        print(f"{ticker}: FILE NOT CREATED")